In [9]:
import os
import time
import dlib
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


os.chdir('/home/hhmt/projects/tma/face_recog/')
train_size = 6407
val_size = 2742
test_size = 0
face_size = 224
project_dimension = 3
alignment_size = face_size * project_dimension
k_neighbors = 1


with open("train_data.pkl", "rb") as f:
    x_train = pickle.load(f)
x_train = x_train.reshape(-1, face_size, face_size, 3)
x_train = x_train[:,:,:,0]

with open("train_labels.pkl", "rb") as f:
    y_train = pickle.load(f)


with open("validation_data.pkl", "rb") as f:
    x_val = pickle.load(f)
x_val = x_val.reshape(-1, face_size, face_size, 3)
x_val = x_val[:,:,:,0]

with open("validation_labels.pkl", "rb") as f:
    y_val = pickle.load(f)


# Calculate mean of all train data
mean = np.zeros((face_size, face_size))
for i in range(0, face_size):
    for j in range(0, face_size):
        for t in range(0, train_size):
            mean[i][j] += x_train[t][i][j]
        mean[i][j] /= train_size


# Generate Scatter Matrix
scatter_matrix = np.zeros((face_size, face_size))
for i in range(0, train_size):
    scatter_matrix += (x_train[i,:,:] - mean).dot((x_train[i,:,:] - mean).T)


# Generate projection vector
projection_vector = np.zeros((face_size, 0))

eigen_values, eigen_vectors = np.linalg.eig(scatter_matrix)
eigen_pairs = [(np.abs(eigen_values[i]), eigen_vectors[:,i]) for i in range(0, face_size)]
eigen_pairs.sort(key=lambda x: x[0], reverse=True)

for i in range(0, project_dimension):
    projection_vector = np.hstack((projection_vector, eigen_pairs[i][1].reshape(face_size, 1)))


# Transform x_train 
tmp = []
for i in range(0, train_size):
    tmp.append(projection_vector.T.dot(x_train[i]))
x_train = np.zeros((train_size, project_dimension, face_size))
x_train = np.concatenate([tmp])
    

# Align x_train to 1D
x_train = x_train.reshape(train_size, alignment_size)


# Transform x_val
tmp = []
for i in range(0, val_size):
    tmp.append(projection_vector.T.dot(x_val[i]))
x_val = np.zeros((val_size, project_dimension, face_size))
x_val = np.concatenate([tmp])


# Align x_val to 1D
x_val = x_val.reshape(val_size, alignment_size)


# 
scaler = StandardScaler()  
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

classifier = KNeighborsClassifier(n_neighbors=k_neighbors)
classifier.fit(x_train, y_train)

y_pred = np.ravel(classifier.predict(x_val))

print(classification_report(y_val, y_pred))  
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1096
           1       0.98      0.98      0.98       228
           2       1.00      0.92      0.96       427
           3       0.96      1.00      0.98       682
           4       0.97      0.99      0.98       309

   micro avg       0.98      0.98      0.98      2742
   macro avg       0.98      0.98      0.98      2742
weighted avg       0.98      0.98      0.98      2742

[[1094    0    0    1    1]
 [   0  223    0    1    4]
 [   2    4  391   25    5]
 [   2    0    0  680    0]
 [   1    0    0    2  306]]
0.9824945295404814


In [16]:
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.18      0.31      1096
           1       1.00      0.02      0.03       228
           2       0.18      1.00      0.31       427
           3       0.97      0.17      0.30       682
           4       0.92      0.22      0.35       309

   micro avg       0.30      0.30      0.30      2742
   macro avg       0.81      0.32      0.26      2742
weighted avg       0.85      0.30      0.29      2742

[[199   0 897   0   0]
 [  2   4 213   4   5]
 [  0   0 426   0   1]
 [  1   0 562 119   0]
 [  1   0 241   0  67]]


In [2]:
print(classification_report(y_val, y_pred))  
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.49      0.63      1096
           1       1.00      0.07      0.13       228
           2       0.23      1.00      0.38       427
           3       0.94      0.29      0.44       682
           4       0.94      0.23      0.37       309

   micro avg       0.46      0.46      0.46      2742
   macro avg       0.80      0.42      0.39      2742
weighted avg       0.81      0.46      0.47      2742

[[540   0 554   2   0]
 [ 33  16 167   8   4]
 [  0   0 426   0   1]
 [  9   0 477 196   0]
 [ 32   0 203   2  72]]
0.45587162654996355


In [4]:
print(classification_report(y_val, y_pred))  
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.98      0.74      1096
           1       1.00      0.03      0.06       228
           2       1.00      0.96      0.98       427
           3       0.87      0.54      0.67       682
           4       0.95      0.25      0.39       309

   micro avg       0.71      0.71      0.71      2742
   macro avg       0.88      0.55      0.57      2742
weighted avg       0.80      0.71      0.66      2742

[[1074    0    0   22    0]
 [ 200    7    0   18    3]
 [  14    0  412    0    1]
 [ 312    0    0  370    0]
 [ 216    0    0   17   76]]
0.7071480671043034


In [6]:
print(classification_report(y_val, y_pred))  
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.97      0.75      1096
           1       1.00      0.06      0.12       228
           2       1.00      0.84      0.91       427
           3       0.82      0.67      0.74       682
           4       0.95      0.26      0.41       309

   micro avg       0.72      0.72      0.72      2742
   macro avg       0.88      0.56      0.59      2742
weighted avg       0.80      0.72      0.68      2742

[[1058    0    0   38    0]
 [ 190   14    0   21    3]
 [  65    0  359    2    1]
 [ 223    0    0  459    0]
 [ 188    0    0   40   81]]
0.7188183807439825


In [8]:
print(classification_report(y_val, y_pred))  
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1096
           1       0.96      0.98      0.97       228
           2       1.00      0.91      0.95       427
           3       0.96      0.99      0.98       682
           4       0.98      0.98      0.98       309

   micro avg       0.98      0.98      0.98      2742
   macro avg       0.98      0.97      0.97      2742
weighted avg       0.98      0.98      0.98      2742

[[1095    0    0    1    0]
 [   1  224    0    0    3]
 [   4   10  388   21    4]
 [   4    0    0  678    0]
 [   3    0    0    3  303]]
0.9803063457330415
